# Laboratorium 5 (4 pkt)

Celem czwartego laboratorium jest zapoznanie się oraz zaimplementowanie algorytmów głębokiego uczenia aktywnego. Zaimplementowane algorytmy będą testowane z wykorzystaniem środowiska z OpenAI - *CartPole*.


Dołączenie standardowych bibliotek

In [31]:
from collections import deque
import gym
import numpy as np
import random
from tqdm import tqdm
from numpy import ndarray

Dołączenie bibliotek do obsługi sieci neuronowych

In [32]:
# import tensorflow as tf
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras import Sequential
# import numpy as np

In [33]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Zadanie 1 - Double Deep Q-Network

<p style='text-align: justify;'>
Celem ćwiczenie jest zaimplementowanie algorytmu Double Deep Q-Network. Wartoscią oczekiwaną sieci jest:
\begin{equation}
       Q^*(s, a) \approx r + \gamma argmax_{a'}Q_\theta'(s', a') 
\end{equation}
a wagi pomiędzy sieciami wymieniane są co dziesięć aktualizacji wag sieci sterującej poczynaniami agenta ($Q$).
</p>

In [34]:
class ReplayBuffer(object):
    def __init__(self, mem_size, state_shape):
        self.mem_size = mem_size
        self.mem_cntr = 0
        self.state_memory = np.zeros((self.mem_size, state_shape), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, state_shape), dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=bool)

    def store_transition(self, state, action, reward, _state, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = _state
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.terminal_memory[index] = 1 - done
        self.mem_cntr += 1

    @staticmethod
    def _softmax(x: ndarray) -> ndarray:
        return np.exp(x) / np.exp(x).sum()

    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)

        probs = self._softmax(np.abs(self.reward_memory[:max_mem]))
        batch = np.random.choice(max_mem, batch_size, p=probs)

        states = self.state_memory[batch]
        actions = self.action_memory[batch]
        rewards = self.reward_memory[batch]
        _states = self.new_state_memory[batch]
        done = self.terminal_memory[batch]

        return states, actions, rewards, _states, done

In [35]:
class DQN(nn.Module):
    def __init__(self, lr, state_shape, n_actions, fc1, fc2):
        super(DQN, self).__init__()

        self.fc1 = nn.Linear(state_shape, fc1)
        self.fc2 = nn.Linear(fc1, fc2)
        self.output = nn.Linear(fc2, n_actions)
        self.optimizer = optim.RMSprop(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        self.device = T.device('cuda:0')
        self.to(self.device)

    def forward(self, state):
        state = F.relu(self.fc1(state))
        state = F.relu(self.fc2(state))
        actions = self.output(state)

        return actions

In [36]:
class DDQNAgent:
    def __init__(self, state_size, action_size, learning_rate):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = ReplayBuffer(5_000, state_size)
        self.gamma = 0.95    # discount rate
        self.epsilon = 0.5  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.05
        self.q = DQN(learning_rate, state_size, action_size, 32, 32)
        self.q_target = DQN(learning_rate, state_size, action_size, 32, 32)
        self.update_weights()
        self.evaluate=False

    def remember(self, state, action, reward, _state, done):
        self.memory.store_transition(state, action, reward, _state, done)

    def get_action(self, state):
        if np.random.random() <= self.epsilon and not self.evaluate:
            action = np.random.choice(self.action_size)
        else:
            state = T.tensor(state).to(self.q.device)
            actions = self.q.forward(state)
            action = T.argmax(actions).item()

        return action


    def get_best_action(self, state):
        state = T.tensor(state).to(self.q.device)
        actions = self.q.forward(state)
        action = T.argmax(actions).item()

        return action

    def replay(self, batch_size):
        """
        Function learn network using randomly selected actions from the memory.
        First calculates Q value for the next state and choose action with the biggest value.
        Target value is calculated according to:
                Q(s,a) := (r + gamma * max_a(Q(s', a)))
        except the situation when the next action is the last action, in such case Q(s, a) := r.
        In order to change only those weights responsible for chosing given action, the rest values should be those
        returned by the network for state state.
        The network should be trained on batch_size samples.
        After each 10 Q Network trainings parameters should be copied to the target Q Network
        """
        if self.memory.mem_cntr < batch_size:
            return

        states, actions, rewards, _states, done = self.memory.sample_buffer(batch_size)

        states = T.tensor(states).to(self.q.device)
        _states = T.tensor(_states).to(self.q_target.device)

        q_next = self.q_target(_states).cpu().detach().numpy()
        q_pred = self.q(states)
        q_target = q_pred.cpu().detach().numpy().copy()

        max_actions = np.argmax(q_next, axis=1)

        batch_index = np.arange(batch_size, dtype=np.int32)

        q_target[batch_index, actions] = rewards + self.gamma * q_next[batch_index, max_actions.astype(int)] * (1-done)
        q_target = T.tensor(q_target).to(self.q.device)

        self.q.optimizer.zero_grad()
        loss = self.q.loss(q_pred, q_target).to(self.q.device)
        loss.backward()
        self.q.optimizer.step()

        if not self.memory.mem_cntr % 10:
            self.update_weights()

    def update_epsilon_value(self):
        self.epsilon = self.epsilon - self.epsilon_decay if self.epsilon > self.epsilon_min else self.epsilon_min

    def update_weights(self):
        self.q_target.load_state_dict(self.q.state_dict())


Czas przygotować model sieci, która będzie się uczyła działania w środowisku [*CartPool*](https://gym.openai.com/envs/CartPole-v0/):

In [37]:
env = gym.make("CartPole-v0")
state_size = env.observation_space.sample().shape[0]
action_size = env.action_space.n
learning_rate = 0.001

Czas nauczyć agenta gry w środowisku *CartPool*:

In [38]:
agent = DDQNAgent(state_size, action_size, learning_rate)

agent.epsilon = 1

done = False
batch_size = 64
EPISODES = 1000
counter = 0
for e in range(EPISODES):
    summary = []
    for _ in tqdm(range(100), desc=f'Epoch: {e}', disable=True):
        total_reward = 0
        state = env.reset()
        state = np.array(state, dtype=np.float32)
        done = False

        for time in range(500):
            action = agent.get_action(state)
            _state, reward, done, _ = env.step(action)
            _state = np.array(_state, dtype=np.float32)
            total_reward += reward

            agent.remember(state, action, reward, _state, done)
            agent.replay(batch_size)
            state = _state
            if done:
                break

        summary.append(total_reward)

    agent.update_epsilon_value()
    print("epoch #{}\tmean reward = {:.3f}\tepsilon = {:.3f}".format(e, np.mean(summary), agent.epsilon))    
    
    if np.mean(summary) > 195:
        print ("You Win!")
        break


epoch #0	mean reward = 22.530	epsilon = 0.950
epoch #1	mean reward = 22.420	epsilon = 0.900
epoch #2	mean reward = 21.270	epsilon = 0.850
epoch #3	mean reward = 19.270	epsilon = 0.800
epoch #4	mean reward = 18.750	epsilon = 0.750
epoch #5	mean reward = 17.580	epsilon = 0.700
epoch #6	mean reward = 15.640	epsilon = 0.650
epoch #7	mean reward = 15.510	epsilon = 0.600
epoch #8	mean reward = 13.150	epsilon = 0.550
epoch #9	mean reward = 11.990	epsilon = 0.500
epoch #10	mean reward = 12.020	epsilon = 0.450
epoch #11	mean reward = 12.280	epsilon = 0.400
epoch #12	mean reward = 11.270	epsilon = 0.350
epoch #13	mean reward = 11.540	epsilon = 0.300
epoch #14	mean reward = 10.770	epsilon = 0.250
epoch #15	mean reward = 10.450	epsilon = 0.200
epoch #16	mean reward = 10.900	epsilon = 0.150
epoch #17	mean reward = 10.250	epsilon = 0.100
epoch #18	mean reward = 9.760	epsilon = 0.050
epoch #19	mean reward = 9.880	epsilon = -0.000
epoch #20	mean reward = 9.420	epsilon = 0.010
epoch #21	mean reward = 9

KeyboardInterrupt: 